# Sesiunea 7 – Proiect OOP complet
_Notebook de exerciții (fără soluții)._

## 🎯 Exercițiul 1 – Definirea clasei `Student`
Creează o clasă `Student` care să conțină:
- Atribute: `nume`, `email`, `note`
- Metode: `adauga_nota()`, `media()`, `__str__()`

Testează crearea unui obiect `Student` și adăugarea notelor.

In [23]:
import inspect

class Student:
    def __init__(self, name, email, grades: list):
        self.name = name
        self.email = email
        self.grades = grades

    def add_grade(self, mark):
        self.grades.append(mark)

    def mean_grades(self):
        return sum(self.grades) / len(self.grades) if len(self.grades) != 0 else 0

    def __str__(self):
        return (f"{self.name.ljust(10)}{self.email.ljust(20)}{self.mean_grades():.2f}")

    @classmethod
    def str_header(cls):
        """Return a formatted table header matching the output of __str__()."""
        return ((f"{'Name'.ljust(10)}{'E-mail'.ljust(20)}{'Grades'}"))

    def __eq__(self, other):
        if self is other: # memory address check, avoids recursion, doesn't call `__eq__` like `==`
            return True
        elif inspect.isclass(other): # values check, other classes, hmmm, canonic
            return vars(self) == vars(other)
        elif isinstance(other, str):
            return self.name.casefold() == other.casefold()
        elif isinstance(other, list):
            return self.grades == other
        return NotImplemented


student_mark = Student("Mark", "mark@email.com", [5, 10, 9])
mar2 = Student("Mar", "mark", [5, 10, 9, 6, 7])
student_mark.add_grade(6)
print(Student.str_header())
print(student_mark)
print(mar2)

Name      E-mail              Grades
Mark      mark@email.com      7.50
Mar       mark                7.40


## 🧑‍🏫 Exercițiul 2 – Definirea clasei `Trainer`
Creează o clasă `Trainer` care **moștenește** `Student` și adaugă:
- Atribut suplimentar: `expertiza`
- Metoda `__str__()` care returnează formatul `[Trainer] nume (expertiza)`

Testează instanțierea unui `Trainer`.

In [24]:
class Teacher(Student):
    def __init__(self, *args, expertise, **kwargs):
        self.expertise = expertise
        super().__init__(*args, **kwargs)

    def __str__(self):
        return f"[Trainer] {self.name} ({self.expertise})"

Kelly = Teacher("Kelly", "kelly@email.com", None, expertise="History")
print(Kelly)

[Trainer] Kelly (History)


## 📚 Exercițiul 3 – Definirea clasei `Curs`
Creează o clasă `Curs` care conține un `Trainer` și o listă de `Student`.
Adaugă metodele principale:
- `adauga_student()`
- `sterge_student()`
- `cauta_student()`
- `cel_mai_bun_student()`
- `afiseaza_studenti()`
- `exporta_studenti()`
- `importa_studenti()`

In [ ]:
import json
from pathlib import Path

class Course:
    def __init__(self, *args, 
                 teacher: Teacher,
                 filepath="sesiunea_07_students.json", 
                 **kwargs):

        self.teacher: Teacher = teacher
        self.students: list[Student] = []
        self.file_path = Path(filepath)

    def add_student(self, name, email, grades=[]):
        self.students.append(Student(name, email, grades))

    def remove_student(self, name=None):
        if self.students.count(name) == 1:
            self.students.remove(name)
        else:
            raise ValueError("Too many found")

    def grade_student(self, name, grade):
        index = self.students.index(name)
        self.students[index].add_grade(grade)

    def search_for_student(self, name):
        if self.students.count(name) == 1:
            return self.students[self.students.index(name)]
        elif self.students.count(name) < 1:
            print("No student found")
        else:
            print("Found multiple entries (refine search):")
            return[student for student in self.students if student == name]

    def get_best_student(self):
        return max([(student, student.mean_grades()) for student in self.students], key=lambda x: x[1])

    def display_teacher(self):
        print(self.teacher)

    def display_students(self):
        print(Student.str_header())
        for student in self.students:
            print(student)

    def export_students(self):
        if not self.file_path.exists():
            self.file_path.touch()  # creates empty file
            print("Created:", self.file_path)

        data = [
            {"name": s.name, "email": s.email, "grades": s.grades}
            for s in self.students
        ]

        with open(self.file_path, 'w') as f:
            json.dump(data, f, indent=4)

    def import_students(self):
        with open(self.file_path, "r") as f:
            data = json.load(f)
        self.students = [Student(**d) for d in data]

course = Course(teacher=Teacher("Dr. Evelyn Ross",
                        "evelyn.ross@academy.org",
                        None,
                        expertise="Data Science"))

course.add_student("Maya", "nova23@mail.net", [5, 10, 7])
course.add_student("Rylan", "rjones@outlook.org", [9, 6, 10, 7, 8])
course.add_student("Tessa", "tessa_04@gmail.com", [8, 8])
course.add_student("Kai", "k.mura@domain.com", [7, 9, 5, 10, 6, 9])
course.add_student("Leo", "leosmith@proton.me", [10, 7, 6, 9, 8])
course.add_student("Leo", "leoduck@duck.email", [6, 5, 7, 8])
course.add_student("test", "", [])

course.remove_student("test")
course.export_students()
course.import_students()

course.display_teacher()
print()
course.display_students()

best_student = course.get_best_student()
print("\nBest Student")
print(best_student[0])
print(course.search_for_student(best_student[0]))
print(course.search_for_student("Kai"))
print(course.search_for_student("tessa_04@gmail.com"))
print()

entries_found = course.search_for_student("leo")

for entry in entries_found:
    print(entry)

[Trainer] Dr. Evelyn Ross (Data Science)

Name      E-mail              Grades
Maya      nova23@mail.net     7.33
Rylan     rjones@outlook.org  8.00
Tessa     tessa_04@gmail.com  8.00
Kai       k.mura@domain.com   7.67
Leo       leosmith@proton.me  8.00
Leo       leoduck@duck.email  6.50

Best Student
Rylan     rjones@outlook.org  8.00
Rylan     rjones@outlook.org  8.00
Kai       k.mura@domain.com   7.67
No student found
None

Found multiple entries (refine search):
Leo       leosmith@proton.me  8.00
Leo       leoduck@duck.email  6.50


## 🧩 Exercițiul 4 – Crearea unui meniu interactiv (CLI)
Construiește un meniu interactiv cu opțiunile:
1. Adaugă student
2. Afișează toți
3. Caută după email
4. Șterge student
5. Notează student
6. Cel mai bun student
7. Export în fișier
8. Încarcă din fișier
9. Ieșire

Meniul trebuie să ruleze într-un `while True` și să apeleze metodele din `Curs`.

In [ ]:
count = 0
def interactive_menu():
    while True:
        if not count % 5:
            course.export_students()
        count += 1
        print("""1. Adaugă student
    2. Afișează toți
    3. Caută după email
    4. Șterge student
    5. Notează student
    6. Cel mai bun student
    7. Export în fișier
    8. Încarcă din fișier
    9. Ieșire""")
        choice = input("Choose: ")
        match choice:
            case '1':
                print()
                student = input("Add comma separated (name, email):").split(',')
                course.add_student(*student)
                print(f"Added {student}")
                print()
            case '2':
                print()
                course.display_students()
                print()
            case '3':
                student = input("Email:")
                print()
                print(course.search_for_student(student.casefold()))
                print()
            case '4':
                print()
                student = input("Entry to remove:")
                course.remove_student()
                print(f"Removed {student}")
                print()
            case '5':
                print()
                student = input("student, grade (comma separated): ").split(',')
                course.grade_student(*student)
                print(f"{student} graded")
                print()
            case '6':
                course.get_best_student()
            case '7':
                course.export_students()
            case '8':
                course.import_students()
            case '9':
                break

## ⚙️ Exercițiul 5 – Testare rapidă
Scrie codul principal care:
- Creează un obiect `Trainer`
- Creează un obiect `Curs`
- Pornește meniul interactiv

```python
if __name__ == "__main__":
    trainer = Trainer(...)
    curs = Curs(..., trainer)
    meniu(curs)
```

In [27]:
if __name__ == "__main__":
    teacher = Teacher("Joe", "joe@email.com", None, expertise="History")
    course = Course(teacher=teacher)
    interactive_menu()

1. Adaugă student
    2. Afișează toți
    3. Caută după email
    4. Șterge student
    5. Notează student
    6. Cel mai bun student
    7. Export în fișier
    8. Încarcă din fișier
    9. Ieșire


## 🧠 Exercițiul 6 – Quiz Recapitulativ
- Cum se face moștenirea în Python?
- Ce este `__str__()`?
- De ce folosim compunerea?
- Cum salvezi într-un fișier CSV?
- Cum apelezi metoda `media()`?

In [2]:
# inheritance is done by putting the parent class in parantheses for the child class

# __str__() is a special class for determining how print shows the object

# composition is using object instances in class attributes instead of inheriting directly

# idk, used json, a comma separarate value would be:
# with open("test.csv", "w") as f:
#     f.writelines("test,test,test")

# callinga method:
# student = Student("test", "test", [5, 6, 7, 8])
# student.mean_grades()

## 🏠 Exercițiul 7 – Temă pentru acasă
- Implementează ștergerea studentului după email
- Completează meniul interactiv complet
- Lucrează cu fișiere CSV pentru:
  - Studenți
  - Note
  - Rapoarte

In [ ]:
# done

## 🚀 Exercițiul 8 – Idei de extindere
- Adaugă autentificare pentru trainer
- Permite evaluarea cursului de către studenți
- Creează rapoarte automate în fișiere text
- Adaugă salvare automată la fiecare 5 acțiuni

In [ ]:
# [x] Instructor Auth (done separately here)
# [ ] Evaluation
# [ ] Reports
# [x] Auto-save 5 actions (done in menu func)

# IMPORTANT: delete instructor_login.json to make new account

import os
import json
from crypto_utils import ensure_secret_key, get_secret_key, hmac_sha256_hex, verify_hmac_sha256_hex

import sys
from pathlib import Path

script_dir = Path(sys.argv[0]).resolve().parent
print(".env saves in", script_dir)

secret_key = ensure_secret_key("COURSE_SECRET_KEY")

# Login/Register
if not os.path.exists("instructor_login.json"):
    print("Register:")

    username_input = input("Username: ")
    password_input = input("Password: ")

    secret_key = get_secret_key("COURSE_SECRET_KEY")

    username_hash = hmac_sha256_hex(username_input, secret_key)
    password_hash = hmac_sha256_hex(password_input, secret_key)

    data = [{"username": username_hash, "password": password_hash}]

    with open("instructor_login.json", 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"Login file created successfully.")
else:
    print("instructor_login.json already exists.")
    print("Login:")
    username_input = input("Username: ")
    password_input = input("Password: ")


with open("instructor_login.json", 'r') as f:
    data = json.load(f)

username_hash = hmac_sha256_hex(username_input, secret_key)
password_hash = hmac_sha256_hex(password_input, secret_key)

found = any(
    acc["username"] == username_hash and acc["password"] == password_hash
    for acc in data
)

print(found)

if found:
    password_ok = True
else:
    password_ok = False

if password_ok:
    print("Acces permis")
else:
    print("Acces respins")

.env saves in /home/mintmainog/workspace/VS Code Workspaces/SkillBrain_Python_Homework_Fork/.venv/lib/python3.12/site-packages
instructor_login.json already exists.
Login:
False
Acces respins
